### This notebook addresses research questions 3 and 6.1

##### Requirements:
* comments.csv
* reference.csv

##### Generates:
* top_tokens_hateful.csv
* top_tokens_reference.csv

In [ ]:
import pandas as pd
import re
from collections import Counter, defaultdict
from tqdm import tqdm

from orange_functions import *

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
c = pd.read_csv('../data/processed/comments.csv')
r = pd.read_csv('../data/processed/reference.csv')

In [ ]:
CUSTOM_FILTERS = [strip_tags, 
                strip_punctuation, 
                strip_multiple_whitespaces, 
                strip_non_alphanum, 
                strip_numeric
            ]

my_stopwords = list(STOPWORDS)
my_stopwords.extend(['ve', 'll', 'com', 'https'])

In [ ]:
# male_words = ['amigo','ape','apes','bastard','bastards','bear',
#                 'beast','beefcake','bloke','blokes','boy','boyfriend','boyfriends','boys','brah',
#                 'bro','brother','brothers','bruh','brute','buck','bud','buddies','buddy',
#                 'buds','bugger','bull','chad','chairman','chap','chum','dad','dads','dawg',
#                 'dick','dickhead','dickheads','dilf','dude','fag','faggot','father','father',
#                 'fella','fellow','fiance','fucker','gent','gent','gentleman','gentlemen',
#                 'grandfather','grandfathers','grandpa','grandpas','grandson','grandsons',
#                 'groom','grooms','guy','guys','he',"he'd","he's",'hero','him','himself',
#                 'his','hombre','hombres','hubby','hunk','husband','husbands','incel','incels',
#                 'king','kings','lad','lads','macho','machos','male','man','mankind','master','men',
#                 "men's",'mensch','mister','mr','muscle','nephew','nephews','papa','pop','prick',
#                 'pricks','priest','prince','run-on','sir','son','sonny','sons','stud','thug',
#                 'thugs','uncle','uncles','virile','waiter','waiters','wanker','widower','widowers']


# female_words = ['actress','actresses','amiga','amigas','aunt','aunts','babe','babes','biddy','bimbo',
#                 'bimbos','bride','brides','butch','chairwoman','chick','chicks','cow','cows',
#                 'cunt','cunts','dame','daughter','daughters','doe','dudess','dudette','dudine',
#                 'dyke','dykes','fag hag','fem','female','feme','femme','fiancee','floozy',
#                 'fusby','gal','girl','girlfriend','girlfriends','girls','goddess','goddesses',
#                 'granddaughter','granddaughters','grandma','grandmas','grandmother','grandmothers',
#                 'hag','hags','hay bag','hay-bag','her','heroine','hers','herself','hoe',
#                 'hoe bag','hoes','honey','hooker','hookers','hussy','jugs','karen','ladies',
#                 'lady','lost rib','lubra','mama','mamas','manhole','milf','mistress','mistresses',
#                 'mivvy','mom','moms','mother','mothers','mrs','ms','muff','niece','nieces',
#                 'priestess','princess','prostitute','prostitutes','pussies','pussy','queen',
#                 'queens','quim','scow','she',"she'd","she's",'shickster','sister','sisters',
#                 'skank','skanks','slut','sluts','spokeswoman','strumpet','tar leather',
#                 'tit','toots','tootsie','tramp','waitress','whore','whores','widow','widows',
#                 'wife','wimmin','wives','woman','womankind','women','women',"women's",'womyn']

# f_df = pd.DataFrame({'identifier': female_words})
# m_df = pd.DataFrame({'identifier': male_words})
# f_df['gender'] = 'female'
# m_df['gender'] = 'male'
# gender_id_df = f_df.append(m_df, ignore_index=True)


# def get_tokens(data):
#     '''
#     tokenize all comments and remove stopwords
#     outputs a single list of tokens
#     '''
#     tokens = []
#     print('tokenizing...',end='\r')
#     for line in data.body.to_list():
#         pre = preprocess_string(line.lower(), CUSTOM_FILTERS)
#         tokens.extend(remove_stopword_tokens(pre, my_stopwords))
#     print(' ')
#     return [t for t in tokens if len(t)>1]


# def get_dict(data, k):
#     '''
#     creates dictionary with 
#     counter of terms for each gender
#     '''
#     top_tokens = defaultdict()
#     print('counting tokens...',end='\r')
#     top_tokens['all'] = Counter(get_tokens(data)).most_common(k)
#     top_tokens['female'] = Counter(get_tokens(data[data.gender=='female'])).most_common(k)
#     top_tokens['male'] = Counter(get_tokens(data[data.gender=='male'])).most_common(k)
#     print(' ')
#     return top_tokens


# def identifiers_dict(data, k):
#     '''
#     creates a dictionary with counter of 
#     gender identifiers for each gender
#     '''
#     global gender_id_df

#     pattern = re.compile('|'.join(gender_id_df.identifier.to_list()))

#     words = []
#     print('finding identifiers...',end='\r')
#     for txt in data.body.to_list():
#         words.extend(re.findall(pattern, txt.lower()))

#     top_tokens = defaultdict()
#     top_tokens['f_terms'] = Counter([w for w in words if w in female_words]).most_common(k)
#     top_tokens['m_terms'] = Counter([w for w in words if w in male_words]).most_common(k)

#     tokens=[]
#     for line in data.body.to_list():
#         pre = preprocess_string(line.lower(), CUSTOM_FILTERS)
#         tokens.extend(remove_stopword_tokens(pre, my_stopwords))

#     top_tokens['both_terms'] = Counter([w for w in tokens if w in gender_id_df.identifier.to_list()]).most_common(k)
#     print(' ')
#     return top_tokens


# def get_top_k_tokens(data, k, gender_id=False):
#     '''
#     creates df with k most frequent tokens 
#     for each gender and for all data
#     '''
#     if gender_id:
#         data_dict = identifiers_dict(data, k)
#     else:
#         data_dict = get_dict(data, k)

#     print('creating the dataframe...',end='\r')
#     for i,key in enumerate(data_dict.keys()):
#         cat_df = pd.DataFrame(data_dict[key])
#         cat_df.insert(0, 'gender', key)
#         if i==0:
#             df = cat_df.copy()
#         else:
#             df = df.append(cat_df, ignore_index=True)
#     df.columns = ['gender', 'token', 'freq']
#     df['rank'] = list(range(k))*3
#     print(' ')
#     return df


# def get_percentage(data):
#     '''
#     takes df with top_k gender identifiers (for all genders) 
#     and calculates percentage of frequency within each gender
#     '''
#     for i,g in enumerate(['f_terms', 'm_terms', 'both_terms']):
#         df = data[data['gender'] == g].copy()
#         df['percentage'] = 100 * df.freq/df.freq.sum()
#         if i==0:
#             perc_id = df.copy()
#         else:
#             perc_id = perc_id.append(df, ignore_index=True)
#         perc_id = perc_id.round(2)
#     return perc_id


#### Question 3

How does language differ based on gendered speech? 

##### Operationalized question
How do the words associated with Reddit comments differ based on the presence of male or female pronouns?


### Top tokens

In [ ]:
c_hate = c[c.davidson_label!=2]
top_hate = get_top_k_tokens(c_hate, 20)
top_hate.to_csv('../data/viz/top_tokens_hateful.csv', index=False)

In [ ]:
top_hate.sample(5)

,gender,token,freq,rank
4,all,whore,18587,4
21,female,like,15613,1
0,all,cunt,96307,0
16,all,cum,6673,16
3,all,fucking,19610,3


In [ ]:
top_reference = get_top_k_tokens(r, 20)
top_reference.to_csv('../data/viz/top_tokens_reference.csv', index=False)

In [ ]:
top_reference.sample(5)

,gender,token,freq,rank
9,all,going,4360,9
17,all,day,3371,17
27,female,way,451,7
14,all,work,3657,14
54,male,things,987,14


#### Question 6.1

How often do the gender identifiers appear in the corpus? 

##### Operationalized question
For each gender identifier, what is its frequency in the corpus and its proportion of the separate totals female or male identifiers?

### Top gender identifier terms

In [ ]:
top_id_hate = get_top_k_tokens(c_hate, 20, gender_id=True)

In [ ]:
top_id_reference = get_top_k_tokens(r, 20, gender_id=True)

In [ ]:
perc_id_hate = get_percentage(top_id_hate)
perc_id_ref = get_percentage(top_id_reference)
perc_id_hate.to_csv('../data/viz/top_gender_id_hate.csv', index=False)
perc_id_ref.to_csv('../data/viz/top_gender_id_ref.csv', index=False)

In [ ]:
perc_id_hate.sample(5)

,gender,token,freq,rank,percentage
58,both_terms,girls,1015,18,0.58
28,m_terms,guy,5672,8,1.48
20,m_terms,he,227842,0,59.37
2,f_terms,she,28295,2,10.21
6,f_terms,pussy,6301,6,2.27


In [ ]:
perc_id_ref.sample(5)

,gender,token,freq,rank,percentage
14,f_terms,sister,329,14,0.42
29,m_terms,ape,1733,9,0.53
0,f_terms,her,44790,0,57.00
59,both_terms,father,251,19,2.12
19,f_terms,aunt,151,19,0.19


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b22dad3f-c925-4cd0-bb81-e22d83bd774f' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>